# LangChain Cookbook 👨‍🍳👩‍🍳

### How to get Embedding and LLM Tokens?
Follow the instructions in Slack Thread: https://rubrikinc.slack.com/archives/C092RDBLN/p1746228804836079


### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

**TLDR**: LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs
2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

### **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. **Speed 🚢** - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. **Community 👥** - Wonderful discord and community support, meet ups, hackathons, etc.

Though LLMs can be straightforward (text-in, text-out) you'll quickly run into friction points that LangChain helps with once you develop more complicated applications.

<img src="./images/langchain_stack.svg" alt="langchain_stack" width="700"/>



### Supported Models
https://python.langchain.com/docs/integrations/llms/

In [55]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

azure_openai_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT", None)
azure_openai_key=os.getenv("AZURE_OPENAI_API_KEY", None)
azure_api_version=os.getenv("AZURE_API_VERSION", None)
azure_deployment=os.getenv("AZURE_DEPLOYMENT", None)
azure_embedding_model=os.getenv("AZURE_EMBEDDING_MODEL", None)

def get_azure_chat_model() -> AzureChatOpenAI:
    return AzureChatOpenAI(
        openai_api_version=azure_api_version,
        azure_endpoint=azure_openai_endpoint,
        openai_api_key=azure_openai_key,
        azure_deployment=azure_deployment,
        temperature=0.7,
    )

def get_azure_embedding_model() -> AzureOpenAIEmbeddings:
    return AzureOpenAIEmbeddings(
        model=azure_embedding_model,
        azure_endpoint=azure_openai_endpoint,
        openai_api_key=azure_openai_key,
        api_version=azure_api_version,
    )

# LangChain Components

## Schema - Nuts and Bolts of working with Large Language Models (LLMs)

### **Text**
The natural language way to interact with LLMs

In [56]:
# You'll be working with simple strings (that'll soon grow in complexity!)
my_text = "What day comes after Friday?"
my_text

'What day comes after Friday?'

### **Chat Messages**
Like text, but specified with a message type (System, Human, AI)

* **System** - Helpful background context that tell the AI what to do
* **Human** - Messages that are intented to represent the user
* **AI** - Messages that show what the AI responded with

For more, see OpenAI's [documentation](https://platform.openai.com/docs/guides/chat/introduction)

In [57]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = get_azure_chat_model()

Now let's create a few messages that simulate a chat experience with a bot

In [58]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)
print(f"Response: {ai_message.content}")

Response: How about a fresh caprese salad with tomatoes, mozzarella, basil, and a drizzle of olive oil?


You can also pass more chat history w/ responses from the AI

In [59]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)
print(f"Response: {ai_message.content}")

Response: Explore the Promenade des Anglais, visit the Old Town (Vieux Nice), and savor fresh seafood at beachside restaurants!


You can also exclude the system message if you want

In [60]:
ai_message = chat.invoke(
    [
        HumanMessage(content="What day comes after Thursday?")
    ]
)
print(f"Response: {ai_message.content}")

Response: The day that comes after Thursday is **Friday**.


### **Documents**
An object that holds a piece of text and metadata (more information about that text)

In [61]:
from langchain.schema import Document

In [62]:
Document(
    page_content="This is my document. It is full of text that I've gathered from other places",
    metadata={
        'my_document_id' : 234234,
        'my_document_source' : "The LangChain Papers",
        'my_document_create_time' : 1680013019
    }
)

Document(metadata={'my_document_id': 234234, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019}, page_content="This is my document. It is full of text that I've gathered from other places")

But you don't have to include metadata if you don't want to

In [63]:
Document(page_content="This is my document. It is full of text that I've gathered from other places")

Document(metadata={}, page_content="This is my document. It is full of text that I've gathered from other places")

## Models - The interface to the AI brains

###  **Language Model**
A model that does text in ➡️ text out!

### **Chat Model**
A model that takes a series of messages and returns a message output

In [64]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = get_azure_chat_model()

In [65]:
ai_message = chat.invoke(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)
print(f"Response: {ai_message.content}")

Response: Why not just flap your arms really hard and hope for the best? If birds can do it, why can't you?


### Function Calling Models

[Function calling models](https://openai.com/blog/function-calling-and-other-api-updates) are similar to Chat Models but with a little extra flavor. They are fine tuned to give structured data outputs.

This comes in handy when you're making an API call to an external service or doing extraction.

In [66]:
# Fix this

chat = get_azure_chat_model()

output = chat(messages=
     [
         SystemMessage(content="You are an helpful AI bot"),
         HumanMessage(content="What’s the weather like in Boston right now?")
     ],
     functions=[
         {
             "name": "get_current_weather",
             "description": "Get the current weather in a given location",
             "parameters": {
                 "type": "object",
                 "properties": {
                     "location": {
                         "type": "string",
                         "description": "The city and state, e.g. San Francisco, CA"
                     },
                     "unit": {
                         "type": "string",
                         "enum": ["celsius", "fahrenheit"]
                     }
                 },
                 "required": ["location"]
             }
         }
     ]
)
print(output.additional_kwargs)

{'function_call': {'arguments': '{"location":"Boston, MA"}', 'name': 'get_current_weather'}, 'refusal': None}


See the extra `additional_kwargs` that is passed back to us? We can take that and pass it to an external API to get data. It saves the hassle of doing output parsing.

### **Text Embedding Model**
Embedding is a means of representing objects like text, images and audio as points in a continuous vector space where the locations of those points in space are semantically meaningful to machine learning (ML) algorithms.

#### Why use Embeddings?
<img src="./images/why_use_embedding.png" alt="why_use_embedding" width="500"/>

#### Mathematics behind Embeddings
<img src="./images/vector.jpg" alt="vector" width="700"/>
<img src="./images/vectors.jpg" alt="vector" width="700"/>
<img src="./images/embeddings.jpg" alt="embeddings" width="700"/>


*BTW: Semantic means 'relating to meaning in language or logic.'*

In [13]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = get_azure_embedding_model()

In [67]:
text = "Hi! It's time for the beach"

In [69]:
text_embedding = embeddings.embed_query(text)
print (f"Here's a sample: {text_embedding[:5]}...")
print (f"Your embedding is length {len(text_embedding)}")

Here's a sample: [-0.02253468707203865, -0.016473732888698578, -0.003123833332210779, -0.04386293143033981, 0.014032116159796715]...
Your embedding is length 3072


# Runnable interface

The Runnable interface is the foundation for working with LangChain components, and it's implemented across many of them, such as [language models](/docs/concepts/chat_models), [output parsers](/docs/concepts/output_parsers), [retrievers](/docs/concepts/retrievers), [compiled LangGraph graphs](
https://langchain-ai.github.io/langgraph/concepts/low_level/#compiling-your-graph) and more.

Related Resources
* The ["Runnable" Interface API Reference](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) provides a detailed overview of the Runnable interface and its methods.
* A list of built-in `Runnables` can be found in the [LangChain Core API Reference](https://python.langchain.com/api_reference/core/runnables.html). Many of these Runnables are useful when composing custom "chains" in LangChain using the [LangChain Expression Language (LCEL)](/docs/concepts/lcel).
:::

## Overview of runnable interface

The Runnable way defines a standard interface that allows a Runnable component to be:

* [Invoked](/docs/how_to/lcel_cheatsheet/#invoke-a-runnable): A single input is transformed into an output.
* [Batched](/docs/how_to/lcel_cheatsheet/#batch-a-runnable): Multiple inputs are efficiently transformed into outputs.
* [Streamed](/docs/how_to/lcel_cheatsheet/#stream-a-runnable): Outputs are streamed as they are produced.
* Inspected: Schematic information about Runnable's input, output, and configuration can be accessed.
* Composed: Multiple Runnables can be composed to work together using [the LangChain Expression Language (LCEL)](/docs/concepts/lcel) to create complex pipelines.

## Prompts - Text generally used as instructions to your model


<img src="./images/anatomy_of_a_prompt.png" alt="anatomy_of_a_prompt" width="500"/>

### **Prompt Template**
An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

Think of it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts

In [70]:
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

llm = get_azure_chat_model()


# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there?

Respond in one short sentence
"""

template = PromptTemplate(
    input_variables=["location"],
    template=template,
)

chain = template | llm | StrOutputParser()
print(chain.invoke(input={"location": "Rome"}))

Explore ancient landmarks like the Colosseum, Vatican City, and Trevi Fountain while indulging in authentic Italian cuisine.


### **Example Selectors**
An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

Check out different types of example selectors [here](https://python.langchain.com/docs/how_to/example_selectors/)

In [71]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = get_azure_chat_model()
embeddings = get_azure_embedding_model()


example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings, 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    
    # This is the number of examples to produce.
    k=2
)

similar_prompt_template = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

chain = similar_prompt_template | llm | StrOutputParser()
chain.invoke(input={"noun": "plant"})

'soil'

### **Output Parsers Method 1: Prompt Instructions & String Parsing**
A helpful way to format the output of a model. Usually used for structured output. LangChain has a bunch more output parsers listed on their [documentation](https://python.langchain.com/docs/modules/model_io/output_parsers).

Two big concepts:

**1. Format Instructions** - A autogenerated prompt that tells the LLM how to format it's response based off your desired result

**2. Parser** - A method which will extract your model's text output into a desired structure (usually json)

In [18]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [19]:
llm = get_azure_chat_model()

In [20]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [21]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [22]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt_template = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

chain = prompt_template | llm | output_parser
chain.invoke(input={"user_input": "welcom to califonya!"})

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

### **Output Parsers Method 2: OpenAI Fuctions**
When OpenAI released function calling, the game changed. This is recommended method when starting out.

They trained models specifically for outputing structured data. It became super easy to specify a Pydantic schema and get a structured output.

There are many ways to define your schema, I prefer using Pydantic Models because of how organized they are. Feel free to reference OpenAI's [documention](https://platform.openai.com/docs/guides/gpt/function-calling) for other methods.

**Example 1: Simple**

Let's get started by defining a simple model for us to extract from.

In [23]:
from pydantic import BaseModel, Field
from typing import Optional, Sequence

class Person(BaseModel):
    """Identifying information about a person."""

    name: str = Field(..., description="The person's name")
    age: int = Field(..., description="The person's age")
    fav_food: Optional[str] = Field(None, description="The person's favorite food")


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person] = Field(..., description="The people in the text")


Then let's create a chain (more on this later) that will do the extracting for us

In [73]:
from langchain.chains.openai_functions import create_structured_output_chain

llm = get_azure_chat_model()

prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="{user_input}"
)

chain = prompt_template | llm.with_structured_output(People)
chain.invoke(
    input={
        "user_input": "Sally is 43, Joey just turned 32 and loves spinach. Caroline is 10 years older than Sally."
    }
)

People(people=[Person(name='Sally', age=43, fav_food=None), Person(name='Joey', age=32, fav_food='spinach'), Person(name='Caroline', age=53, fav_food=None)])

Let's do some more parsing with it

**Example 2: Enum**

Now let's parse when a product from a list is mentioned

In [25]:
import enum

llm = get_azure_chat_model()

class Product(str, enum.Enum):
    CRM = "CRM"
    VIDEO_EDITING = "VIDEO_EDITING"
    HARDWARE = "HARDWARE"


class Products(BaseModel):
    """Identifying products that were mentioned in a text"""

    products: Sequence[Product] = Field(..., description="The products mentioned in a text")


In [26]:
chain = prompt_template | llm.with_structured_output(Products)
chain.invoke(
    input={
        "user_input": "The CRM in this demo is great. Love the hardware. The microphone is also cool. Love the video editing"
    }
)

Products(products=[<Product.CRM: 'CRM'>, <Product.VIDEO_EDITING: 'VIDEO_EDITING'>, <Product.HARDWARE: 'HARDWARE'>])

## Indexes - Structuring documents to LLMs can work with them

### **Document Loaders**
Easy ways to import data from other sources. Shared functionality with [OpenAI Plugins](https://openai.com/blog/chatgpt-plugins) [specifically retrieval plugins](https://github.com/openai/chatgpt-retrieval-plugin)

See a [big list](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) of document loaders here. A bunch more on [Llama Index](https://llamahub.ai/) as well.

**HackerNews**

In [74]:
from langchain.document_loaders import HNLoader

In [75]:
loader = HNLoader("https://news.ycombinator.com/item?id=34422627")

In [76]:
data = loader.load()

In [77]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")

Found 76 comments
Here's a sample:

Ozzie_osman on Jan 18, 2023  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are veOzzie_osman on Jan 18, 2023  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_ind


**URLs and webpages**

Let's try it out with [Paul Graham's website](http://www.paulgraham.com/)

In [78]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "http://www.paulgraham.com/",
]

loader = UnstructuredURLLoader(urls=urls)

data = loader.load()

data[0].page_content

'New: What to Do | Wokeness | Founder Mode Want to start a startup? Get funded by Y Combinator . © mmxxv pg'

### **Text Splitters**
Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.

There are many ways you could split your text into chunks, experiment with [different ones](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) to see which is best for you.

In [79]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [83]:
# This is a long document we can split up.
with open('data/PaulGrahamEssays/worked.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")
print(pg_work)

You have 1 document
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what
beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,
just characters with strong feelings, which I imagined made them
deep.The first programs I tried writing were on the IBM 1401 that our
school district used for what was then called "data processing."
This was in 9th grade, so I was 13 or 14. The school district's
1401 happened to be in the basement of our junior high school, and
my friend Rich Draves and I got permission to use it. It was like
a mini Bond villain's lair down there, with all these alien-looking
machines  CPU, disk drives, printer, card reader  sitting up
on a raised floor under bright fluorescent lights.The language we used was an early version of Fortran. You had to
type programs on punch cards, then stack them in the car

In [84]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

In [85]:
print (f"You have {len(texts)} documents")

You have 610 documents


In [86]:
print ("Preview:")
print (texts[0].page_content, "\n")
print (texts[1].page_content)

Preview:
February 2021Before college the two main things I worked on, outside of school,
were writing and programming. I didn't write essays. I wrote what 

beginning writers were supposed to write then, and probably still
are: short stories. My stories were awful. They had hardly any plot,


There are a ton of different ways to do text splitting and it really depends on your retrieval strategy and application design. Check out more splitters [here](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

### **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [87]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('data/PaulGrahamEssays/worked.txt')
documents = loader.load()

In [88]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = get_azure_embedding_model()

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [89]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [40]:
retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11bc34610>, search_kwargs={})

In [90]:
docs = retriever.invoke("what types of things did the author want to build?")

In [91]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

much of it in grad school.Computer Science is an uneasy alliance between two halves, theory
and systems. The theory people prove things, and the systems people
build things. I wanted to build things. 


### **VectorStores**

Databases to store vectors.

#### How do they differ from traditional database?
<img src="./images/vector_database_difference.jpg" alt="vector_database_difference" width="500"/>


Conceptually, think of them as tables w/ a column for embeddings (vectors) and a column for metadata.

Example

| Embedding      | Metadata |
| ----------- | ----------- |
| [-0.00015641732898075134, -0.003165106289088726, ...]      | {'date' : '1/2/23}       |
| [-0.00035465431654651654, 1.4654131651654516546, ...]   | {'date' : '1/3/23}        |

In [92]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('data/PaulGrahamEssays/worked.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = get_azure_embedding_model()

In [93]:
print (f"You have {len(texts)} documents")

You have 78 documents


In [94]:
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

In [95]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 78 embeddings
Here's a sample of one: [0.023125724866986275, -0.0014271973632276058, -0.02350175380706787]...


Your vectorstore store your embeddings (☝️) and make them easily searchable

## Memory
Helping LLMs remember information.

Memory is a bit of a loose term. It could be as simple as remembering information you've chatted about in the past or more complicated information retrieval.

We'll keep it towards the Chat Message use case. This would be used for chat bots.

There are many types of memory, explore [the documentation](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) to see which one fits your use case.

### Chat Message History

In [47]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = get_azure_chat_model()

history = ChatMessageHistory()

history.add_ai_message("hi!")
history.add_user_message("what is the capital of france?")

In [48]:
ai_response = chat.invoke(history.messages)
print(f"Response: {ai_response.content}")

Response: The capital of France is **Paris**.


In [49]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={})]

## Chains ⛓️⛓️⛓️
Combining different LLM calls and action automatically.

### Simple Sequential Chains

Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [50]:
from langchain.prompts import PromptTemplate

llm = get_azure_chat_model()

In [96]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = prompt_template | llm | StrOutputParser()

In [97]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = prompt_template | llm

In [98]:
overall_chain = location_chain | meal_chain

In [99]:
review = overall_chain.invoke(input={"user_location": "Rome"})
print(f"Response:\n{review.content}")

Response:
**Simple Recipe for Cacio e Pepe**

**Ingredients:**
- 12 oz (340g) spaghetti or tonnarelli
- 1 cup Pecorino Romano cheese, finely grated
- 2 tsp freshly ground black pepper (adjust to taste)
- Salt (for boiling pasta)

**Instructions:**
1. **Cook the pasta:** Bring a large pot of salted water to a boil. Cook the pasta until *just al dente*. Reserve about 1 cup of the pasta water before draining.

2. **Toast the pepper:** In a large skillet or pan, toast the black pepper over medium heat for 1-2 minutes to release its aroma.

3. **Create the sauce:** Lower the heat and add about ½ cup of the reserved pasta water to the skillet with the pepper. Gradually stir in the Pecorino Romano cheese, a little at a time, until a creamy sauce forms. Keep stirring to prevent clumping.

4. **Combine pasta and sauce:** Add the drained pasta to the skillet, tossing it with the sauce. If the mixture seems too dry, add a bit more pasta water until the sauce fully coats the pasta.

5. **Serve:** 